Step 2 for pre-processed MOSA WRF data.
* Track
* Classify CEs
* Create df with info but not full set of CE coordinates

In [ ]:
from pathlib import Path

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

#from lib import run_wrf_preproced

## Get files

In [ ]:
files = sorted(Path("/glade/scratch/zmoon/mosa-pre").glob("tb_rainrate*.parquet"))

print(f"{len(files)} total files")

# files = files[:1000]  # test
files = [p for p in files if p.name.startswith("tb_rainrate_2010") or p.name.startswith("tb_rainrate_2011")]  # first WY

print(f"using {len(files)} files")

print(files[0])
print("...")
print(files[-1])

## Run

In [ ]:
ce = run_wrf_preproced(files)
ce

## Plot

In [ ]:
ce.area_km2.astype(np.float64)

In [ ]:
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

fig, ax = plt.subplots(subplot_kw=dict(projection=proj), figsize=(7, 10))

ax.coastlines()
ax.gridlines(draw_labels=True)

ax.scatter(ce.lon, ce.lat, s=ce.area_km2.astype(np.float64)/10000, marker=".", transform=tran, alpha=0.2, lw=0)

## Save df

They only want MCSs, so we can drop the CEs that are not part of an MCS

In [ ]:
mcs = (
    ce[ce.is_mcs]
    .reset_index(drop=True)
    .drop(columns=["is_mcs", "not_is_mcs_reason"])
)
mcs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 9))

n = 0
for _, g in mcs.groupby("mcs_id"):
    g.plot(x="lon", y="lat", ax=ax, label="", legend=False)
    n += 1
    
    if n > 50:
        break

## A function to pre-process GPM data 

In [ ]:
ds=xr.open_mfdataset("/glade/campaign/mmm/c3we/prein/SouthAmerica/MCS-Tracking/GPM/2019/merg_2019123101*.nc",
                     concat_dim='xtime', combine='nested', parallel=True) 

In [ ]:
ds.precipitationCal

In [ ]:
ds.Tb.isel(time=1).plot()

## For IMERG (rain = precipitationCal): 

In [ ]:
## For multiple files
#precip_hourly = [ds.precipitationCal.isel(xtime=i).isel(time=0) for i in range(len(ds.precipitationCal))]

## Function for just one file at a time 
def imerg_hourly(ds):
    precip_hourly = ds.precipitationCal.isel(time=0)
    assert precip_hourly.time.dt.minute == 0
    precip_hourly["time"] = precip_hourly.time.dt.floor("H")
    return (precip_hourly)

## For MERGIR (brightnes temp = Tb):

In [ ]:
## For multiple files
#precip_hourly = [ds.precipitationCal.isel(xtime=i).isel(time=0) for i in range(len(ds.precipitationCal))]

## Function for just one file at a time 
def mergir_hourly(ds):
    Tb_hourly = ds.precipitationCal.mean(dim = "time") 
    Tb_hourly = Tb_hourly.assign_coords(time=ds.time.values[0])
    assert Tb_hourly.time.dt.minute == 0
    Tb_hourly["time"] = Tb_hourly.time.dt.floor("H")
    return Tb_hourly

In [ ]:
mergir_hourly(ds)

In [ ]:
(ds.precipitationCal.isel(time=0)- ds.precipitationCal.isel(time=1)).plot() 

In [ ]:
ds.precipitationCal.isel(time=0)

In [ ]:
precip_hourly = []
for i in range(len(ds.precipitationCal)):
    
    precip_hourly.append(ds.precipitationCal.isel(time=0)  
    